In [1]:
from preproc_tabular import get_tabular
from dataset_sales import getDataset
import sales_prediction as sp
from torch.optim.adamw import AdamW
from torch import nn
import torch
import os

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\ResizedImages\\'
tab_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\sales_anagrafica_final.csv'
aggregated_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_season.csv'
aggregated_month_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_month.csv'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")
modello = sp.create_model(0).to(device)

batch_size = 32
epochs = 5
criterion = nn.MSELoss()
optimizer = AdamW(modello.parameters(), lr=0.01) 
early_stop = sp.EarlyStopping(2, 0.001)

data, references, descriptions, target = get_tabular(tab_path)
train, val = getDataset(references, data, descriptions, target, img_path, batch_size, 0.08)

mse_final, mae_final, r2_final = 0, 0, 0
modello_final = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    sp.train_loop(train, modello, criterion, optimizer, batch_size, device)
    mse, mae = sp.validation_loop(val, modello, criterion, device)
    early_stp = early_stop(mse)
    print(early_stp)
    if early_stp:
        print('Early Stop attivato.')
        break
    mse_final, mae_final = mse, mae
    modello_final = modello
    
print("Done!")

os.makedirs('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\no_agg')
with open('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\no_agg\\descrizione.txt', 'w') as file:
    file.write("Addestramento con dati non aggregati adamw, batch 32, 5 epoche, lr 1e-2, early stopping con 1 epoca e delta 0.001, dropout 0.5." + '\n\n')
    file.write(f"Metriche alla fine:\nMSE: {mse_final}\nMAE: {mae_final}\n")
model_sc = torch.jit.script(modello_final)
model_sc.save('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\no_agg\\weights.pt')

c:\Users\GRVRLD00P\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sto usando cuda


c:\Users\GRVRLD00P\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


Epoch 1
-------------------------------


c:\Users\GRVRLD00P\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


MSE: 1.899285  [   32/111298]
MSE: 2.442423  [ 3232/111298]
MSE: 0.546048  [ 6432/111298]
MSE: 1.900445  [ 9632/111298]
MSE: 1.114205  [12832/111298]
MSE: 1.098116  [16032/111298]
MSE: 0.711382  [19232/111298]
MSE: 0.987636  [22432/111298]
MSE: 0.967865  [25632/111298]
MSE: 0.990417  [28832/111298]
MSE: 1.067485  [32032/111298]
MSE: 1.161305  [35232/111298]
MSE: 1.456364  [38432/111298]
MSE: 1.884824  [41632/111298]
MSE: 0.800128  [44832/111298]
MSE: 0.615805  [48032/111298]
MSE: 0.601667  [51232/111298]
MSE: 0.549368  [54432/111298]
MSE: 1.956987  [57632/111298]
MSE: 0.985540  [60832/111298]
MSE: 2.596916  [64032/111298]
MSE: 2.966568  [67232/111298]
MSE: 0.923467  [70432/111298]
MSE: 1.607583  [73632/111298]
MSE: 0.522065  [76832/111298]
MSE: 1.812312  [80032/111298]
MSE: 1.623674  [83232/111298]
MSE: 3.812880  [86432/111298]
MSE: 1.543065  [89632/111298]
MSE: 0.657890  [92832/111298]
MSE: 1.328916  [96032/111298]
MSE: 0.970183  [99232/111298]
MSE: 1.282972  [102432/111298]
MSE: 0.47

KeyboardInterrupt: 

In [ ]:
import pred_tabular as pt
import sales_prediction as sp
from preproc_tabular import get_tabular
from torch.optim.adamw import AdamW
from torch import nn
import torch
import os

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\ResizedImages\\'
tab_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\sales_anagrafica_final.csv'
aggregated_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_season.csv'
aggregated_month_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_month.csv'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")

modello = pt.create_model(0).to(device)

batch_size = 64
epochs = 10
criterion = nn.MSELoss()
optimizer = AdamW(modello.parameters(), lr=0.1) 
early_stop = sp.EarlyStopping(1, 0.001) 

data, _ , _, target = get_tabular(tab_path)
train, val = pt.getDataset(data, target, batch_size, 0.08)

mse_final, mae_final, r2_final = 0, 0, 0
modello_final = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    pt.train_loop(train, modello, criterion, optimizer, batch_size, device)
    mse, mae, r2 = pt.validation_loop(val, modello, criterion, device)
    early_stp = early_stop(mse)
    print(early_stp)
    if early_stp:
        print('Early Stop attivato.')
        break
    mse_final, mae_final, r2_final = mse, mae, r2
    modello_final = modello
    
print("Done!")

#os.makedirs('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\Tabular_Agg')
with open('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\Tab_NoAgg\\descrizione.txt', 'w') as file:
    file.write("Addestramento con dati non aggregati 0.08 utilizzando solo i tabular, adamw, batch 64, 10 epoche, lr 1e-2, early stopping con 1 epoca e delta 0.001." + '\n\n')
    file.write(f"Metriche alla fine:\nMSE: {mse_final}\nMAE: {mae_final}\nR2: {r2_final}")